# Matching Strings by Similarity (Levenshtein Distance)
Answer each numbered task below.

### Load the two CSVs

In [13]:
%pip install "thefuzz[speedup]" -q

Note: you may need to restart the kernel to use updated packages.


In [14]:

import pandas as pd
import itertools 
from thefuzz import fuzz, process    # pip install thefuzz[speedup] if needed

df1 = pd.read_csv('companies_1.csv')
df2 = pd.read_csv('companies_2.csv')
df1.head(), df2.head()


(                                  CLIENT
 0                    Adobe Systems, Inc.
 1                       Adventist Health
 2                                  AECOM
 3   Aerojet Rockedyne Holdings (GenCorp)
 4  Alameda-Contra Costa Transit District,
                                            Firm Name
 0  AAA Northern California, Nevada & Utah Auto Ex...
 1                            ACCO Engineered Systems
 2                       Adams County Retirement Plan
 3                               Adidas America, Inc.
 4                                Adobe Systems, Inc.)

## 1️⃣  Create `df` with the cartesian product (`CSV 1`, `CSV 2`)

In [17]:
product = itertools.product(df1['CLIENT'], df2['Firm Name'])
cartesian_df = pd.DataFrame(product, columns=['CSV 1','CSV 2'])

In [20]:
cartesian_df.head()

,CSV 1,CSV 2
0,"Adobe Systems, Inc.","AAA Northern California, Nevada & Utah Auto Ex..."
1,"Adobe Systems, Inc.",ACCO Engineered Systems
2,"Adobe Systems, Inc.",Adams County Retirement Plan
3,"Adobe Systems, Inc.","Adidas America, Inc."
4,"Adobe Systems, Inc.","Adobe Systems, Inc."


In [22]:
cartesian_df.shape

(97888, 2)

In [ ]:
## **Calculating the Levenshtein distance** 

## 2️⃣  Add `Ratio Score` column using `fuzz.partial_ratio`

In [38]:
score = [fuzz.partial_ratio(c1, c2) for c1, c2 in zip(cartesian_df['CSV 1'], cartesian_df['CSV 2'])]

In [39]:
score[:10]

[32, 64, 41, 50, 100, 59, 29, 35, 54, 60]

In [33]:
cartesian_df['Ratio Score'] = score

In [35]:
cartesian_df.head(10)

,CSV 1,CSV 2,Ratio Score
0,"Adobe Systems, Inc.","AAA Northern California, Nevada & Utah Auto Ex...",32
1,"Adobe Systems, Inc.",ACCO Engineered Systems,64
2,"Adobe Systems, Inc.",Adams County Retirement Plan,41
3,"Adobe Systems, Inc.","Adidas America, Inc.",50
4,"Adobe Systems, Inc.","Adobe Systems, Inc.",100
5,"Adobe Systems, Inc.","Advanced Micro Devices, Inc.",59
6,"Adobe Systems, Inc.",AECOM Technology Corporation,29
7,"Adobe Systems, Inc.",Aera Energy LLC,35
8,"Adobe Systems, Inc.","Aerojet Rocketdyne Holdings, Inc.",54
9,"Adobe Systems, Inc.","Agilent Technologies, Inc.",60


In [41]:
cartesian_df.shape

(97888, 3)

## 3️⃣  How many rows have a **Ratio Score ≥ 90**?

In [48]:
cartesian_df.loc[cartesian_df['Ratio Score'] >= 90].head()

,CSV 1,CSV 2,Ratio Score
4,"Adobe Systems, Inc.","Adobe Systems, Inc.",100
742,AECOM,AECOM Technology Corporation,100
1112,Aerojet Rockedyne Holdings (GenCorp),"Aerojet Rocketdyne Holdings, Inc.",90
1484,Alameda-Contra Costa Transit District,Alameda-Contra Costa Transit District,100
3697,Amazon,"Amazon.com Holdings, Inc.",100


In [75]:
cartesian_df.loc[cartesian_df['Ratio Score'] >= 90].shape

(135, 3)

## 4️⃣  What’s the corresponding company in **CSV 2** to **AECOM** in **CSV 1**?

In [55]:
cartesian_df.query("`CSV 1` == 'AECOM' and `Ratio Score` > 80")

,CSV 1,CSV 2,Ratio Score
742,AECOM,AECOM Technology Corporation,100


## 5️⃣  What’s the corresponding **CSV 2** company of **Starbucks**?

In [56]:
cartesian_df.query("`CSV 1` == 'Starbucks' and `Ratio Score` > 80")

,CSV 1,CSV 2,Ratio Score
77948,Starbucks,Starbucks Corporation,100


## 6️⃣  Is there a matching company (>90) for **Pinnacle West Capital Corporation** in **CSV 2**?

In [57]:
cartesian_df.query("`CSV 1` == 'Pinnacle West Capital Corporation' and `Ratio Score` > 90")

,CSV 1,CSV 2,Ratio Score
61130,Pinnacle West Capital Corporation,Ball Corporation,93


## 7️⃣  How many matching companies are there for **County of Los Angeles Deferred Compensation Program**?

In [72]:
cartesian_df.query("`CSV 1` == 'County of Los Angeles Deferred Compensation Program' and `Ratio Score` > 90")

,CSV 1,CSV 2,Ratio Score
26206,County of Los Angeles Deferred Compensation Pr...,City of Los Angeles Deferred Compensation,95
26227,County of Los Angeles Deferred Compensation Pr...,County of Los Angeles Deferred Compensation Pr...,100


In [73]:
cartesian_df.query("`CSV 1` == 'County of Los Angeles Deferred Compensation Program' and `Ratio Score` > 90").count()

CSV 1          2
CSV 2          2
Ratio Score    2
dtype: int64

## 8️⃣  Is there a matching company (>90) for **The Queens Health Systems**? (Yes/No)

In [74]:
cartesian_df.query("`CSV 1` == 'The Queens Health Systems' and `Ratio Score` > 90")

,CSV 1,CSV 2,Ratio Score
84220,The Queens Health Systems,The Queen's Health Systems,96
